**INTRODUÇÃO**

Neste notebook, acessamos a API do YouTube para coletar informações sobre os vídeos de Whindersson Nunes, como visualizações, curtidas e comentários ao longo dos anos, e de seu pseudônimo, Lil Whind.

**ÍNDICE**

1. [Consulta à API](#Consulta)

  1.1 [Instalação de bibliotecas e autenticação](#Instalacao)


2. [Whindersson Nunes](#Whindersson)

  2.1 [Estatísticas do canal](#Estatisticas-Whindersson)

  2.2 [Informações de playlists e vídeos](#Playlists-Whindersson)
  
  2.3 [Informações sobre comentários, likes e visualizações](#Analise-Whindersson)


3. [Lil Whind](#Lil-Whind)

  3.1 [Estatísticas do canal](#Estatisticas-Lil)

  3.2 [Informações de playlists e vídeos](#Playlists-Lil)
  
  3.3 [Informações sobre comentários, likes e visualizações](#Analise-Lil)

In [1]:
path = 'dataYoutube/'

## 1. Consulta à API <a name="Consulta"></a>

### 1.1 Instalação de bibliotecas e autenticação <a name="Instalacao"></a>

In [ ]:
!pip install --upgrade google-api-python-client

In [2]:
from pandas import pandas as pd
from googleapiclient.discovery import build

In [61]:
youTubeApiKey = 'youTubeApiKey'

In [4]:
youtube = build('youtube', 'v3', developerKey=youTubeApiKey)

## 2. Whindersson Nunes <a name="Whindersson"></a>

### 2.1 Estatísticas do canal <a name="Estatisticas-Whindersson"></a>

In [5]:
# id do canal do Whindersson Nunes

channel_id = 'UC3KQ5GWANYF8lChqjZpXsQw'

In [6]:
# informações sobre canal

canalWhindersson = youtube.channels().list(part=['snippet','statistics'], id=channel_id).execute()

In [7]:
canalWhindersson

{'kind': 'youtube#channelListResponse',
 'etag': 'Mn8vuZYX3lRDu1M29sEZ1eeemqo',
 'pageInfo': {'totalResults': 1, 'resultsPerPage': 5},
 'items': [{'kind': 'youtube#channel',
   'etag': 'ImRO6Sde1H8boDrUrv8_HbB6GIc',
   'id': 'UC3KQ5GWANYF8lChqjZpXsQw',
   'snippet': {'title': 'whinderssonnunes',
    'description': 'Lampião do Youtube\nPublicidade / Shows / Eventos: kaka@nonstopproducoes.com.br',
    'customUrl': '@whindersson',
    'publishedAt': '2013-01-21T13:06:10Z',
    'thumbnails': {'default': {'url': 'https://yt3.ggpht.com/DOwHDrnH5l4IYq9yGJ0EhxdctKnQUHgzvzJPUy6m-B8-TMruCi2HcQ4HTh5cT1F-zUnr-oPj=s88-c-k-c0x00ffffff-no-rj',
      'width': 88,
      'height': 88},
     'medium': {'url': 'https://yt3.ggpht.com/DOwHDrnH5l4IYq9yGJ0EhxdctKnQUHgzvzJPUy6m-B8-TMruCi2HcQ4HTh5cT1F-zUnr-oPj=s240-c-k-c0x00ffffff-no-rj',
      'width': 240,
      'height': 240},
     'high': {'url': 'https://yt3.ggpht.com/DOwHDrnH5l4IYq9yGJ0EhxdctKnQUHgzvzJPUy6m-B8-TMruCi2HcQ4HTh5cT1F-zUnr-oPj=s800-c-k-c0x00ff

In [8]:
estatisticas_canal = pd.DataFrame.from_dict(canalWhindersson['items'][0]['statistics'], orient='index').reset_index().rename(columns={'index': 'estatísticas', 0: 'valores'})

In [9]:
estatisticas_canal

,estatísticas,valores
0,viewCount,4193902478
1,subscriberCount,44000000
2,hiddenSubscriberCount,False
3,videoCount,527


### 2.2 Informações de playlists e vídeos <a name="Playlists-Whindersson"></a>

In [10]:
# coletar informações das playlists do canal

nextPageToken = None

playlists_whindersson = []

while True:
    resultado = youtube.playlists().list(part=['snippet'], channelId=channel_id, pageToken=nextPageToken).execute()
    
    playlists_whindersson += resultado['items']
    
    nextPageToken = resultado.get('nextPageToken')
    
    if nextPageToken is None:
        break

# criar lista com os Ids das playlists

ids_playlists = list(map(lambda x: x['id'], playlists_whindersson))
nome_playlists = list(map(lambda x: x['snippet']['title'], playlists_whindersson))

# criar dataframe com informações de id e nome das playlists
info_playlists = dict(zip(ids_playlists, nome_playlists))
df_info_playlists = pd.DataFrame.from_dict(info_playlists, orient='index').reset_index().rename(columns={'index': 'id_playlist', 0:'nome_playlist'})

In [11]:
df_info_playlists.head()

,id_playlist,nome_playlist
0,PLypClbHDpCW0dYckCViQEEBj2ztjG545I,VEDA
1,PLypClbHDpCW3UZ-zEkSBWqNwpL_VbVCTH,Lil Whind - Frente & Verso
2,PLypClbHDpCW1H_VYaQvn1wplR2bUrmnnp,Whindersson: Próxima Parada S2
3,PLypClbHDpCW2O_RQYYs-sy4_xvfy_b6b1,Whindersson: Próxima Parada
4,PLypClbHDpCW3-wt36qfTiwF-qW4UDDvpW,PARÓDIAS!


In [12]:
# coletar informações dos vídeos das playlists

nextPageToken = None

itens_playlists = []

for playlist in ids_playlists:
    while True:
        videos_playlists = youtube.playlistItems().list(part='snippet', playlistId=playlist, pageToken=nextPageToken).execute()
        itens_playlists += videos_playlists['items']
        
        nextPageToken = videos_playlists.get('nextPageToken')
        
        if nextPageToken is None:
            break

# criar lista com Ids de cada vídeo
ids_videos = list(map(lambda x: x['snippet']['resourceId']['videoId'], itens_playlists))
nomes_videos = list(map(lambda x: x['snippet']['title'], itens_playlists))
videos_playlist_id = list(map(lambda x: x['snippet']['playlistId'], itens_playlists))

In [13]:
# criar dataframe com informações sobre vídeos e playlists

infos_videos_playlist = list(zip(nomes_videos, ids_videos, videos_playlist_id))
df_infos_videos_playlist = pd.DataFrame.from_records(infos_videos_playlist, columns=['titulo_video', 'id_video', 'id_playlist'])
df_infos_videos_playlist_novo = df_infos_videos_playlist.merge(df_info_playlists, on='id_playlist', how='left')

In [14]:
df_infos_videos_playlist_novo.head()

,titulo_video,id_video,id_playlist,nome_playlist
0,VEDA # 01 - MINHA CASA EM FORTAL,d616HI34WsI,PLypClbHDpCW0dYckCViQEEBj2ztjG545I,VEDA
1,VEDA # 02 - BASTIDORES E ESTREIA EM FORTAL,JXH31F0Rook,PLypClbHDpCW0dYckCViQEEBj2ztjG545I,VEDA
2,"VEDA # 03 - MEU AP EM FORTALEZA, FUSCÃO PRETO",L4NsV3_R45A,PLypClbHDpCW0dYckCViQEEBj2ztjG545I,VEDA
3,"VEDA # 04 - REGINA CHEGOU, PRÉVIAS DO LIL WHIND",z7nAQVbr5eY,PLypClbHDpCW0dYckCViQEEBj2ztjG545I,VEDA
4,VEDA # 05 - RACHA DO NATHANZIN,w3PFJrkkZl4,PLypClbHDpCW0dYckCViQEEBj2ztjG545I,VEDA


In [15]:
# criar lista com dados sobre cada vídeo

lista_dados_video = []

for id_video in ids_videos:
    dados_video_whindersson = youtube.videos().list(part=['snippet', 'statistics', 'player', 'recordingDetails'], id=id_video).execute()
    lista_dados_video += dados_video_whindersson['items']

dados_video_whindersson = []

for dados_video in lista_dados_video:
    for k, v in dados_video.items():
        titulo_video = dados_video['snippet']['title']
        id_video = dados_video['id']
        data_publicacao = dados_video['snippet']['publishedAt']
        imagem = dados_video['snippet']['thumbnails']['medium']['url']
        for k, v in dados_video['player'].items():
            player_video = dados_video['player']['embedHtml']
            for k, v in dados_video['statistics'].items():
                estatisticas = dados_video['statistics']
                if k == 'commentCount':
                    comentarios = dados_video['statistics']['commentCount']
                elif k == 'favoriteCount':
                    favoritos = dados_video['statistics']['favoriteCount']
                elif k == 'likeCount':
                    likes = dados_video['statistics']['likeCount']
                elif k == 'viewCount':
                    visualizacoes = dados_video['statistics']['viewCount']
    dados_video_whindersson.append((titulo_video, id_video, data_publicacao, imagem, player_video, estatisticas, comentarios, favoritos, likes, visualizacoes))

In [16]:
# dataframe com dados dos vídeos do canal de Lil Whind

df_videos = pd.DataFrame.from_records(dados_video_whindersson, columns=['titulo_video', 'id_video', 'data_publicacao', 'imagem', 'player_video', 'estatisticas', 'comentarios', 'favoritos', 'likes', 'visualizacoes'])
df_videos_novo = df_videos.merge(df_infos_videos_playlist_novo, on=['titulo_video', 'id_video'], how='left')

In [17]:
df_videos_novo.to_csv(path + 'whindersson_videos.csv', encoding='utf-8', index=False)

In [18]:
df_videos_novo.head(2)

,titulo_video,id_video,data_publicacao,imagem,player_video,estatisticas,comentarios,favoritos,likes,visualizacoes,id_playlist,nome_playlist
0,VEDA # 01 - MINHA CASA EM FORTAL,d616HI34WsI,2022-04-02T06:52:05Z,https://i.ytimg.com/vi/d616HI34WsI/mqdefault.jpg,"<iframe width=""480"" height=""270"" src=""//www.yo...","{'viewCount': '1980986', 'likeCount': '354357'...",9094,0,354357,1980986,PLypClbHDpCW0dYckCViQEEBj2ztjG545I,VEDA
1,VEDA # 02 - BASTIDORES E ESTREIA EM FORTAL,JXH31F0Rook,2022-04-03T05:52:56Z,https://i.ytimg.com/vi/JXH31F0Rook/mqdefault.jpg,"<iframe width=""480"" height=""270"" src=""//www.yo...","{'viewCount': '586945', 'likeCount': '116395',...",3219,0,116395,586945,PLypClbHDpCW0dYckCViQEEBj2ztjG545I,VEDA


### 2.3 Informações sobre comentários, likes e visualizações <a name="Analise-Whindersson"></a>

In [19]:
df_videos_novo['comentarios'] = df_videos_novo['comentarios'].astype(int)
df_videos_novo['likes'] = df_videos_novo['likes'].astype(int)
df_videos_novo['visualizacoes'] = df_videos_novo['visualizacoes'].astype(int)

In [20]:
df_videos_sem_duplicacao = df_videos_novo.drop_duplicates(subset='id_video')

In [21]:
df_videos_sem_duplicacao['publicacao'] = df_videos_sem_duplicacao['data_publicacao'].str.slice(start=0, stop=10)
df_videos_sem_duplicacao['publicacao'] = pd.to_datetime(df_videos_sem_duplicacao['publicacao'], format='%Y-%m-%d')
df_videos_sem_duplicacao['ano_publicacao'] = df_videos_sem_duplicacao['publicacao'].dt.year

In [22]:
df_videos_sem_duplicacao.head(2)

,titulo_video,id_video,data_publicacao,imagem,player_video,estatisticas,comentarios,favoritos,likes,visualizacoes,id_playlist,nome_playlist,publicacao,ano_publicacao
0,VEDA # 01 - MINHA CASA EM FORTAL,d616HI34WsI,2022-04-02T06:52:05Z,https://i.ytimg.com/vi/d616HI34WsI/mqdefault.jpg,"<iframe width=""480"" height=""270"" src=""//www.yo...","{'viewCount': '1980986', 'likeCount': '354357'...",9094,0,354357,1980986,PLypClbHDpCW0dYckCViQEEBj2ztjG545I,VEDA,2022-04-02,2022
1,VEDA # 02 - BASTIDORES E ESTREIA EM FORTAL,JXH31F0Rook,2022-04-03T05:52:56Z,https://i.ytimg.com/vi/JXH31F0Rook/mqdefault.jpg,"<iframe width=""480"" height=""270"" src=""//www.yo...","{'viewCount': '586945', 'likeCount': '116395',...",3219,0,116395,586945,PLypClbHDpCW0dYckCViQEEBj2ztjG545I,VEDA,2022-04-03,2022


In [23]:
# TOP 10 - vídeos com mais likes
top_10_likes = df_videos_sem_duplicacao.sort_values(by='likes', ascending=False)
top_10_likes = top_10_likes.head(10)
df_top_10_likes = top_10_likes[['titulo_video', 'id_video', 'player_video', 'likes']]
df_top_10_likes = df_top_10_likes.rename(columns={'likes':'Likes'})
df_top_10_likes_tidy = pd.melt(df_top_10_likes, id_vars=['titulo_video', 'id_video', 'player_video'], value_vars='likes').rename(columns={'titulo_video': 'Vídeo', 'player_video': 'Assista', 'variable': 'Estatística', 'value': 'Quantidade'})

In [24]:
df_top_10_likes_tidy.to_csv(path + 'whindersson_top_10_likes.csv', index=False)

In [25]:
# TOP 10 - vídeos com mais visualizações
top_10_visualizacoes = df_videos_sem_duplicacao.sort_values(by='visualizacoes', ascending=False).head(10)
top_10_visualizacoes = top_10_visualizacoes[['titulo_video', 'id_video', 'imagem', 'player_video', 'visualizacoes','publicacao', 'ano_publicacao']]
top_10_visualizacoes = top_10_visualizacoes.rename(columns={'visualizacoes':'Visualizações'})
df_top_10_visualizacoes_tidy = pd.melt(top_10_visualizacoes, id_vars=['titulo_video', 'id_video', 'player_video'], value_vars='Visualizações').rename(columns={'titulo_video': 'Vídeo', 'player_video': 'Assista', 'variable': 'Estatística', 'value': 'Quantidade'})

In [26]:
df_top_10_visualizacoes_tidy.to_csv(path + 'whindersson_top_10_visualizacoes.csv', index=False)

In [27]:
# TOP 10 - vídeos com mais comentários
top_10_comentarios = df_videos_sem_duplicacao.sort_values(by='comentarios', ascending=False)
df_top_10_comentarios = top_10_comentarios.head(10)
df_top_10_comentarios = df_top_10_comentarios[['titulo_video', 'id_video', 'imagem', 'player_video', 'comentarios','publicacao', 'ano_publicacao']]
df_top_10_comentarios = df_top_10_comentarios.rename(columns={'comentarios':'Comentários'})
df_top_10_comentarios_tidy = pd.melt(df_top_10_comentarios, id_vars=['titulo_video', 'id_video', 'player_video'], value_vars='Comentários').rename(columns={'titulo_video': 'Vídeo', 'player_video': 'Assista', 'variable': 'Estatística', 'value': 'Quantidade'})

In [28]:
df_top_10_comentarios_tidy.to_csv(path + 'whindersson_top_10_comentarios.csv', index=False)

Concatenar os três dataframes de top 10 em formato tidy

In [29]:
df_top_10_total = pd.concat([df_top_10_likes_tidy, df_top_10_visualizacoes_tidy, df_top_10_comentarios_tidy])
df_top_10_total = df_top_10_total[['Vídeo', 'id_video', 'Assista', 'Estatística', 'Quantidade']]

In [30]:
df_top_10_total.to_csv(path + 'whindersson_top_10_total.csv', index=False)

Estatísticas por ano

In [31]:
estatisticas_por_ano = df_videos_sem_duplicacao[['ano_publicacao', 'comentarios', 'likes', 'visualizacoes']].groupby(by='ano_publicacao').sum().reset_index().rename(columns={'ano_publicacao': 'Ano de publicação', 'comentarios': 'Comentários', 'likes': 'Likes', 'visualizacoes': 'Visualizações'})
df_estatisticas_por_ano = estatisticas_por_ano.melt(id_vars=['Ano de publicação'], value_vars=['Comentários', 'Likes', 'Visualizações']).rename(columns={'variable': 'Estatística', 'value': 'Quantidade'})

In [32]:
df_estatisticas_por_ano.to_csv(path + 'whindersson_estatisticas_por_ano_canal.csv', index=False)

## 3. Lil Whind <a name="Lil-Whind"></a>

### 3.1 Estatísticas do canal <a name="Estatisticas-Lil"></a>

In [33]:
# id do canal do Lil Whind

channel_id_lil_whind = 'UCPGxg_uxXrheuHX6HrQ-OnA'

In [34]:
# informações sobre canal

canalLilWhind = youtube.channels().list(part=['snippet','statistics'], id=channel_id_lil_whind).execute()

In [35]:
canalLilWhind

{'kind': 'youtube#channelListResponse',
 'etag': 'uPwRfhkEkesPwzHVW2JXl9N0pTU',
 'pageInfo': {'totalResults': 1, 'resultsPerPage': 5},
 'items': [{'kind': 'youtube#channel',
   'etag': 'pufVgVq8MEUFNyFdekMPsk8g1-o',
   'id': 'UCPGxg_uxXrheuHX6HrQ-OnA',
   'snippet': {'title': 'Lil Whind',
    'description': '*CANAL OFICIAL*\n\nLil’ Whind vem do Piauí para o mundo explodindo no TRAP. Musicalidade que não se prende, personalidade forte, batidas envolventes, intensas, loucas, sinestésicas, quânticas. \nNos seus versos cabem entretenimento, cuscuz, alegrias, desilusões, mais cuscuz, sua evolução e a nova realidade que ele cria pra si.\nLil whind tem os pés bem firmes e a mente onde bem entende, um choque da sua própria realidade na realidade de quem ousa segui-lo.\nVocês estão de capacete?!',
    'customUrl': '@lilwhindoficial',
    'publishedAt': '2020-09-20T13:39:20.823254Z',
    'thumbnails': {'default': {'url': 'https://yt3.ggpht.com/yE8HG6O76dGCtRNcl23dlvUDnYpV8om2zP8B1oKWQ4RYdX-qRN22

In [36]:
estatisticas_canal_lil = pd.DataFrame.from_dict(canalLilWhind['items'][0]['statistics'], orient='index').reset_index().rename(columns={'index': 'estatísticas', 0: 'valores'})

In [37]:
estatisticas_canal_lil

,estatísticas,valores
0,viewCount,67250397
1,subscriberCount,631000
2,hiddenSubscriberCount,False
3,videoCount,23


### 3.2 Informações de playlists e vídeos <a name="Playlists-Lil"></a>

In [38]:
# coletar informações das playlists do canal

nextPageToken = None

playlists_lil = []

while True:
    resultado = youtube.playlists().list(part=['snippet'], channelId=channel_id_lil_whind, pageToken=nextPageToken).execute()
    
    playlists_lil += resultado['items']
    
    nextPageToken = resultado.get('nextPageToken')
    
    if nextPageToken is None:
        break

# criar lista com os Ids das playlists

ids_playlists_lil = list(map(lambda x: x['id'], playlists_lil))
nome_playlists_lil = list(map(lambda x: x['snippet']['title'], playlists_lil))

# criar dataframe com informações de id e nome das playlists
info_playlists_lil = dict(zip(ids_playlists_lil, nome_playlists_lil))
df_info_playlists_lil = pd.DataFrame.from_dict(info_playlists_lil, orient='index').reset_index().rename(columns={'index': 'id_playlist', 0:'nome_playlist'})

In [39]:
df_info_playlists_lil

,id_playlist,nome_playlist
0,PLIciEJ3HUZBUfcHpLyG46DQB4QHX7lrwc,PIAUÍ
1,PLIciEJ3HUZBXMCyNO04EETNf5QVdkIP8I,FRENTE & VERSO


In [40]:
# coletar informações dos vídeos das playlists

nextPageToken = None

itens_playlist_lil = []

for playlist in ids_playlists_lil:
    while True:
        videos_playlists = youtube.playlistItems().list(part='snippet', playlistId=playlist, pageToken=nextPageToken).execute()
        itens_playlist_lil += videos_playlists['items']
        
        nextPageToken = videos_playlists.get('nextPageToken')
        
        if nextPageToken is None:
            break

# criar lista com Ids de cada vídeo
ids_videos_lil = list(map(lambda x: x['snippet']['resourceId']['videoId'], itens_playlist_lil))
nomes_videos_lil = list(map(lambda x: x['snippet']['title'], itens_playlist_lil))
videos_playlist_id_lil = list(map(lambda x: x['snippet']['playlistId'], itens_playlist_lil))

In [41]:
# criar dataframe com informações sobre vídeos e playlists

infos_videos_playlist_lil = list(zip(nomes_videos_lil, ids_videos_lil, videos_playlist_id_lil))
df_infos_videos_playlist_lil = pd.DataFrame.from_records(infos_videos_playlist_lil, columns=['titulo_video', 'id_video', 'id_playlist'])
df_infos_videos_playlist_lil_novo = df_infos_videos_playlist_lil.merge(df_info_playlists_lil, on='id_playlist', how='left')

In [42]:
df_infos_videos_playlist_lil_novo

,titulo_video,id_video,id_playlist,nome_playlist
0,Lil Whind - Tempo,d_onQZuBd7c,PLIciEJ3HUZBUfcHpLyG46DQB4QHX7lrwc,PIAUÍ
1,Lil Whind - Caribe,YVRSqEWh20s,PLIciEJ3HUZBUfcHpLyG46DQB4QHX7lrwc,PIAUÍ
2,Lil Whind - Mãe,kdpGcyRteHg,PLIciEJ3HUZBUfcHpLyG46DQB4QHX7lrwc,PIAUÍ
3,Lil Whind - Cerrado,nyaw7FfSMTk,PLIciEJ3HUZBUfcHpLyG46DQB4QHX7lrwc,PIAUÍ
4,Lil Whind - Cuscuz,GqutEnlLX6o,PLIciEJ3HUZBUfcHpLyG46DQB4QHX7lrwc,PIAUÍ
5,Lil Whind - Calor do Carai (feat. Hungria Hip ...,lKJ3UYDKOl0,PLIciEJ3HUZBUfcHpLyG46DQB4QHX7lrwc,PIAUÍ
6,Lil Whind - Amor Quântico,KE7Do_JxDOY,PLIciEJ3HUZBUfcHpLyG46DQB4QHX7lrwc,PIAUÍ
7,Lil Whind - Cachorro Caramelo (feat. Matheus G...,qjVD3cX-Wqc,PLIciEJ3HUZBUfcHpLyG46DQB4QHX7lrwc,PIAUÍ
8,Lil Whind - Catinga de Bosta,cODPJ9E2tDM,PLIciEJ3HUZBUfcHpLyG46DQB4QHX7lrwc,PIAUÍ
9,Lil Whind - Trap do Gago,0kJV1Ctiubk,PLIciEJ3HUZBUfcHpLyG46DQB4QHX7lrwc,PIAUÍ


### 3.3  Informações sobre comentários, likes e visualizações <a name="Analise-Lil"></a>

In [43]:
# criar lista com dados sobre cada vídeo

lista_dados_video_lil = []

for id_video in ids_videos_lil:
    dados_video_lil = youtube.videos().list(part=['snippet', 'statistics', 'player', 'recordingDetails'], id=id_video).execute()
    lista_dados_video_lil += dados_video_lil['items']

dados_video_lil_whind = []

for dados_video in lista_dados_video_lil:
    for k, v in dados_video.items():
        titulo_video = dados_video['snippet']['title']
        id_video = dados_video['id']
        data_publicacao = dados_video['snippet']['publishedAt']
        imagem = dados_video['snippet']['thumbnails']['medium']['url']
        for k, v in dados_video['player'].items():
            player_video = dados_video['player']['embedHtml']
        for k, v in dados_video['statistics'].items():
            estatisticas = dados_video['statistics']
            if k == 'commentCount':
                comentarios = dados_video['statistics']['commentCount']
            elif k == 'favoriteCount':
                favoritos = dados_video['statistics']['favoriteCount']
            elif k == 'likeCount':
                likes = dados_video['statistics']['likeCount']
            elif k == 'viewCount':
                visualizacoes = dados_video['statistics']['viewCount']
    dados_video_lil_whind.append((titulo_video, id_video, data_publicacao, imagem, player_video, estatisticas, comentarios, favoritos, likes, visualizacoes))

In [44]:
# dataframe com dados dos vídeos do canal de Lil Whind

df_videos_lil = pd.DataFrame.from_records(dados_video_lil_whind, columns=['titulo_video', 'id_video', 'data_publicacao', 'imagem', 'player_video', 'estatisticas', 'comentarios', 'favoritos', 'likes', 'visualizacoes'])
df_videos_lil_novo = df_videos_lil.merge(df_infos_videos_playlist_lil_novo, on=['titulo_video', 'id_video'], how='left')

In [45]:
df_videos_lil_novo['publicacao'] = pd.to_datetime(df_videos_lil_novo['data_publicacao']).dt.strftime('%d/%m/%Y')

In [46]:
newDF_videos_lil_novo = df_videos_lil_novo[['titulo_video', 'publicacao', 'imagem', 'player_video', 'comentarios', 'favoritos', 'likes', 'visualizacoes', 'nome_playlist']]

In [47]:
def objToNum(df, col):
    df[col] = df[col].astype(int)

In [48]:
objToNum(newDF_videos_lil_novo, 'comentarios')
objToNum(newDF_videos_lil_novo, 'favoritos')
objToNum(newDF_videos_lil_novo, 'likes')
objToNum(newDF_videos_lil_novo, 'visualizacoes')

C:\Users\red.gabrielacustodio\AppData\Local\Temp\ipykernel_9984\1292008531.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = df[col].astype(int)


In [49]:
newDF_videos_lil_novo.to_csv(path + 'lil_whind_videos.csv', index=False)

In [50]:
videos_lil_whind_sem_duplicacao = newDF_videos_lil_novo.drop_duplicates()

In [51]:
videos_lil_whind_sem_duplicacao.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18 entries, 0 to 17
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   titulo_video   18 non-null     object
 1   publicacao     18 non-null     object
 2   imagem         18 non-null     object
 3   player_video   18 non-null     object
 4   comentarios    18 non-null     int32 
 5   favoritos      18 non-null     int32 
 6   likes          18 non-null     int32 
 7   visualizacoes  18 non-null     int32 
 8   nome_playlist  18 non-null     object
dtypes: int32(4), object(5)
memory usage: 1.1+ KB


In [52]:
# TOP 10 - vídeos com mais likes
videos_lil_whind_sem_duplicacao = newDF_videos_lil_novo.drop_duplicates()
lil_whind_top_10_likes = videos_lil_whind_sem_duplicacao.sort_values(by='likes', ascending=False).head(10)
df_lil_whind_top_10_likes = lil_whind_top_10_likes[['titulo_video', 'player_video', 'likes']]
df_lil_whind_top_10_likes = df_lil_whind_top_10_likes.rename(columns={'likes':'Likes', 'titulo_video': 'Título', 'player_video': 'Player'})

In [53]:
df_lil_whind_top_10_likes

,Título,Player,Likes
10,Lil Whind - Piauí (feat. Rapadura),"<iframe width=""480"" height=""270"" src=""//www.yo...",475888
9,Lil Whind - Trap do Gago,"<iframe width=""480"" height=""270"" src=""//www.yo...",416454
5,Lil Whind - Calor do Carai (feat. Hungria Hip ...,"<iframe width=""480"" height=""270"" src=""//www.yo...",232142
7,Lil Whind - Cachorro Caramelo (feat. Matheus G...,"<iframe width=""480"" height=""270"" src=""//www.yo...",141741
2,Lil Whind - Mãe,"<iframe width=""480"" height=""270"" src=""//www.yo...",132199
3,Lil Whind - Cerrado,"<iframe width=""480"" height=""270"" src=""//www.yo...",112872
0,Lil Whind - Tempo,"<iframe width=""480"" height=""270"" src=""//www.yo...",111726
4,Lil Whind - Cuscuz,"<iframe width=""480"" height=""270"" src=""//www.yo...",91606
1,Lil Whind - Caribe,"<iframe width=""480"" height=""270"" src=""//www.yo...",91320
6,Lil Whind - Amor Quântico,"<iframe width=""480"" height=""270"" src=""//www.yo...",72659


In [54]:
df_lil_whind_top_10_likes.to_csv(path + 'lil_whind_top10_likes.csv', index=False)

In [55]:
# TOP 10 - vídeos com mais comentarios
lil_whind_top_10_comentarios = videos_lil_whind_sem_duplicacao.sort_values(by='comentarios', ascending=False).head(10)
df_lil_whind_top_10_comentario = lil_whind_top_10_comentarios[['titulo_video', 'player_video', 'comentarios']]
df_lil_whind_top_10_comentario = df_lil_whind_top_10_comentario.rename(columns={'comentarios':'Comentários', 'titulo_video': 'Título', 'player_video': 'Player'})

In [56]:
df_lil_whind_top_10_comentario

,Título,Player,Comentários
10,Lil Whind - Piauí (feat. Rapadura),"<iframe width=""480"" height=""270"" src=""//www.yo...",15000
9,Lil Whind - Trap do Gago,"<iframe width=""480"" height=""270"" src=""//www.yo...",12380
5,Lil Whind - Calor do Carai (feat. Hungria Hip ...,"<iframe width=""480"" height=""270"" src=""//www.yo...",5744
3,Lil Whind - Cerrado,"<iframe width=""480"" height=""270"" src=""//www.yo...",3054
7,Lil Whind - Cachorro Caramelo (feat. Matheus G...,"<iframe width=""480"" height=""270"" src=""//www.yo...",2941
2,Lil Whind - Mãe,"<iframe width=""480"" height=""270"" src=""//www.yo...",2800
1,Lil Whind - Caribe,"<iframe width=""480"" height=""270"" src=""//www.yo...",2437
0,Lil Whind - Tempo,"<iframe width=""480"" height=""270"" src=""//www.yo...",2361
4,Lil Whind - Cuscuz,"<iframe width=""480"" height=""270"" src=""//www.yo...",2209
8,Lil Whind - Catinga de Bosta,"<iframe width=""480"" height=""270"" src=""//www.yo...",1713


In [57]:
df_lil_whind_top_10_comentario.to_csv(path + 'lil_whind_top10_comentario.csv', index=False)

In [58]:
# TOP 10 - vídeos com mais visualizações
lil_whind_top_10_visualizacoes = videos_lil_whind_sem_duplicacao.sort_values(by='visualizacoes', ascending=False).head(10)
df_lil_whind_top_10_visualizacoes = lil_whind_top_10_visualizacoes[['titulo_video', 'player_video', 'visualizacoes']]
df_lil_whind_top_10_visualizacoes = df_lil_whind_top_10_visualizacoes.rename(columns={'visualizacoes':'Visualizações', 'titulo_video': 'Título', 'player_video': 'Player'})

In [59]:
df_lil_whind_top_10_visualizacoes

,Título,Player,Visualizações
10,Lil Whind - Piauí (feat. Rapadura),"<iframe width=""480"" height=""270"" src=""//www.yo...",17833676
9,Lil Whind - Trap do Gago,"<iframe width=""480"" height=""270"" src=""//www.yo...",10470274
5,Lil Whind - Calor do Carai (feat. Hungria Hip ...,"<iframe width=""480"" height=""270"" src=""//www.yo...",4528208
7,Lil Whind - Cachorro Caramelo (feat. Matheus G...,"<iframe width=""480"" height=""270"" src=""//www.yo...",3592240
2,Lil Whind - Mãe,"<iframe width=""480"" height=""270"" src=""//www.yo...",3572555
3,Lil Whind - Cerrado,"<iframe width=""480"" height=""270"" src=""//www.yo...",2641489
1,Lil Whind - Caribe,"<iframe width=""480"" height=""270"" src=""//www.yo...",2584982
0,Lil Whind - Tempo,"<iframe width=""480"" height=""270"" src=""//www.yo...",2208929
4,Lil Whind - Cuscuz,"<iframe width=""480"" height=""270"" src=""//www.yo...",1740151
6,Lil Whind - Amor Quântico,"<iframe width=""480"" height=""270"" src=""//www.yo...",1659062


In [60]:
df_lil_whind_top_10_visualizacoes.to_csv(path + 'lil_whind_top10_visualizacoes.csv', index=False)